In [6]:
import os
from PIL import Image
import torch

from diffusion.controlnet import ControlNetModel
from diffusion.pipeline_controlnet import StableDiffusionControlNetPipeline
from diffusers import UniPCMultistepScheduler
from diffusers.utils import load_image

device = "cuda:0"

### Set model paths

In [9]:
base_model_path = "runwayml/stable-diffusion-v1-5"      # NOTE: set as the base Stable Diffusion model
controlnet_path = "./runs/controlnet_fill50k_lr1e-5_batch8_acc1_epochs3"           # NOTE: set as the output directory of your ControlNet training

### Load models and scheduler

In [ ]:
# Load ControlNet
controlnet = ControlNetModel.from_pretrained(
    controlnet_path, 
    torch_dtype=torch.float16
)
# Load Stable Diffusion with ControlNet
pipe = StableDiffusionControlNetPipeline.from_pretrained(
    base_model_path, 
    controlnet=controlnet, 
    torch_dtype=torch.float16,
    safety_checker=None,
)
# Set scheduler
pipe.scheduler = UniPCMultistepScheduler.from_config(pipe.scheduler.config)

# Load model to GPU
pipe.to(device)

### Load condition for ControlNet

In [ ]:
control_image = load_image("./data/test_conditions/4.jpg")
prompt = "forest green circle with an antique brown background"

print(f"Prompt: {prompt}")
control_image

In [ ]:
seed = 10
generator = torch.manual_seed(seed)

# Generate image
image = pipe(
    prompt, 
    num_inference_steps=20, 
    generator=generator, 
    image=control_image
).images[0]

image.save("./output4.png")
image


In [ ]:
# test prompts
# {
#     "0": "pale golden rod circle with old lace background",
#     "1": "sea green circle with a light cyan background",
#     "2": "deep sky blue circle with a light yellow background",
#     "3": "rosy brown circle with a misty rose background",
#     "4": "forest green circle with an antique brown background"
# }

# test conditioning images
# task_1_controlnet/data/test_conditions/0.png
# task_1_controlnet/data/test_conditions/1.png
# task_1_controlnet/data/test_conditions/2.jpg
# task_1_controlnet/data/test_conditions/3.jpg
# task_1_controlnet/data/test_conditions/4.jpg

# generate images
# seed = 10
# generator = torch.manual_seed(seed)

test_prompts = [
    # "pale golden rod circle with old lace background",
    "sea green circle with a light cyan background",
    "deep sky blue circle with a light yellow background",
    # "rosy brown circle with a misty rose background",
    # "forest green circle with an antique brown background"
]
test_conditioning_images = [
    # "./data/test_conditions/0.png",
    "./data/test_conditions/1.png",
    "./data/test_conditions/2.jpg",
    # "./data/test_conditions/3.jpg",
    # "./data/test_conditions/4.jpg"
]

for i, (prompt, conditioning_image) in enumerate(zip(test_prompts, test_conditioning_images)):
    control_image = load_image(conditioning_image)
    print(f"Prompt: {prompt}")
    # control_image.show()
    image = pipe(
        prompt, 
        num_inference_steps=20, 
        generator=generator, 
        image=control_image
    ).images[0]
    # image.save(os.path.join(controlnet_path, f"./output{i}.png"))
    paired_plot = Image.new("RGB", (control_image.width * 2, control_image.height))
    paired_plot.paste(control_image, (0, 0))
    paired_plot.paste(image, (control_image.width, 0))
    paired_plot.save(os.path.join(controlnet_path, f"./paired_plot{i}.png"))
    paired_plot.show()
    image.save(os.path.join(controlnet_path, f"./output{i}.png"))

## Optional

Load Model and Scheduler for ControlNet

In [ ]:
base_model_path = "runwayml/stable-diffusion-v1-5"      # NOTE: set as the base Stable Diffusion model
controlnet_path = "./runs/controlnet_fill50k_50k_lrlinear_2epochs_batch4"           # NOTE: set as the output directory of your ControlNet training

In [ ]:
# Load ControlNet
controlnet = ControlNetModel.from_pretrained(
    controlnet_path, 
    torch_dtype=torch.float16
)
# Load Stable Diffusion with ControlNet
pipe = StableDiffusionControlNetPipeline.from_pretrained(
    base_model_path, 
    controlnet=controlnet, 
    torch_dtype=torch.float16,
    safety_checker=None,
)
# Set scheduler
pipe.scheduler = UniPCMultistepScheduler.from_config(pipe.scheduler.config)

# Load model to GPU
pipe.to(device)

Load Images

In [ ]:
control_images = [
    'optional_data/image0.png',
    'optional_data/image1.png',
    'optional_data/image2.png',
    'optional_data/image3.png',
    'optional_data/image4.png'
]

text_prompts = [
    "Cars driving down a ancient roman street",
"Fancy luxury cars are on the roads",
"There are people marching on the road",
"Military truck, tanks are parked on the roads",
"People are walking in the city of paris"
]

seed = 10
generator = torch.manual_seed(seed)

# image looks like: control image, generated image and below the images, the prompt.

for i, (prompt, conditioning_image) in enumerate(zip(text_prompts, control_images)):
    control_image = load_image(conditioning_image)
    print(f"Prompt: {prompt}")
    # control_image.show()
    image = pipe(
        prompt, 
        num_inference_steps=20, 
        generator=generator, 
        image=control_image
    ).images[0]
    # image.save(os.path.join(controlnet_path, f"./output{i}.png"))
    paired_plot = Image.new("RGB", (control_image.width * 2, control_image.height))
    paired_plot.paste(control_image, (0, 0))
    paired_plot.paste(image, (control_image.width, 0))
    paired_plot.save(os.path.join(controlnet_path, f"./optional-paired_plot{i}.png"))
    paired_plot.show()
    image.save(os.path.join(controlnet_path, f"./optional-output{i}.png"))